In [1]:
%jsroot
// 打开 heat.root 文件
// TFile *heatFile = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106/heat.root");
// TFile *heatFile = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106/light.root");
TFile *heatFile = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/SB13_T6at8p5mK_bkg_9h20min_5kHz_2408232128/heat.root");
TTree *heatTree = (TTree*)heatFile->Get("tree1"); 

// 打开 light.root 文件
// TFile *lightFile = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106/light.root");
// TFile *lightFile = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106/heat.root");
TFile *lightFile = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/SB13_T6at8p5mK_bkg_9h20min_5kHz_2408232128/light.root");
TTree *lightTree = (TTree*)lightFile->Get("tree1"); 

// 定义变量存储 MaxPos 信息
Long64_t heatMaxPos, lightMaxPos;

// 设置分支地址
heatTree->SetBranchAddress("trigpos", &heatMaxPos);
lightTree->SetBranchAddress("trigpos", &lightMaxPos);

// // 创建用于存储 light 信号分布的直方图
// // TH1F *hLightDistribution = new TH1F("hLightDistribution", "Light Events around Heat Signal", 100, -50, 50);  // 100 bins, x范围为-50到50
// TH1F *hLightDistribution = new TH1F("hLightDistribution", "Light Events around Heat Signal", 150, -50, 100);  // 150 bins, x范围为-50到100

// 提取 heat 和 light 事件的 MaxPos 信息到向量
std::vector<Long64_t> heatMaxPosList, lightMaxPosList;

Long64_t nHeatEntries = heatTree->GetEntries();
Long64_t nLightEntries = lightTree->GetEntries();

// 提取 heat 文件中的 MaxPos 列表
for (Long64_t i = 0; i < nHeatEntries; ++i) {
    heatTree->GetEntry(i);
    heatMaxPosList.push_back(heatMaxPos);
}

// 提取 light 文件中的 MaxPos 列表
for (Long64_t i = 0; i < nLightEntries; ++i) {
    lightTree->GetEntry(i);
    lightMaxPosList.push_back(lightMaxPos);
}

int fillCount = 0; // 记录填充次数

In [2]:
// 创建用于存储 light 信号分布的直方图
// TH1F *hLightDistribution = new TH1F("hLightDistribution", "Light Events around Heat Signal", 100, -50, 50);  // 100 bins, x范围为-50到50
TH1F *hLightDistribution = new TH1F("hLightDistribution", "Light Events around Heat Signal", 40, 10, 50);  // 150 bins, x范围为-50到100
// 遍历每个 heat 事件，查找对应的 light 事件
for (size_t i = 0; i < heatMaxPosList.size(); ++i) {
    Long64_t currentHeatMaxPos = heatMaxPosList[i];

    // 对每个 heat 事件，查找附近 (±50) 的 light 事件
    for (size_t j = 0; j < lightMaxPosList.size(); ++j) {
        Long64_t currentLightMaxPos = lightMaxPosList[j];
        Long64_t delta = currentLightMaxPos - currentHeatMaxPos;

        // 如果 light 事件在 heat 事件的±50范围内
        // if (delta >= -50 && delta <= 50) {
        if (delta >= -50 && delta <= 100) {
            hLightDistribution->Fill(delta);  // 将光信号的时间差填入直方图
            fillCount++; // 计数
        }
    }
}

std::cout << "Number of filled bins: " << fillCount << std::endl;

// 定义高斯函数并指定拟合范围为20到40
TF1 *gausFit = new TF1("gausFit", "gaus", 20, 40);

// 对直方图进行高斯拟合
hLightDistribution->Fit(gausFit, "R");  // "R" 表示在指定范围内拟合

// 绘制 light 分布的直方图
TCanvas *canvas = new TCanvas("canvas", "Light Events Distribution", 800, 600);
hLightDistribution->GetXaxis()->SetTitle("Time difference (Light MaxPos - Heat MaxPos)");
hLightDistribution->GetYaxis()->SetTitle("Counts");
canvas->cd();
hLightDistribution->Draw();
gausFit->Draw("same");  // 绘制高斯拟合结果
// canvas->Draw();
// 保存图像（可选）
canvas->SaveAs("Light_Distribution.png");
// 关闭文件
heatFile->Close();
lightFile->Close();

Number of filled bins: 6058
 FCN=46.4966 FROM MIGRAD    STATUS=CONVERGED      69 CALLS          70 TOTAL
                     EDM=7.66598e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   3.1 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     8.00276e+01   3.38993e+00   5.93318e-03  -1.27838e-05
   2  Mean         3.05549e+01   2.90824e-01   2.33525e-04  -7.16265e-05
   3  Sigma        7.12386e+00   4.36084e-01  -3.81130e-05   5.27569e-03


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
Info in <TCanvas::Print>: png file Light_Distribution.png has been created


In [3]:
// 创建用于存储 light 信号分布的直方图
// TH1F *hLightDistribution = new TH1F("hLightDistribution", "Light Events around Heat Signal", 100, -50, 50);  // 100 bins, x范围为-50到50
TH1F *hLightDistribution = new TH1F("hLightDistribution", "Light Events around Heat Signal", 150, -100, 50);  // 150 bins, x范围为-50到100
// 遍历每个 heat 事件，查找对应的 light 事件
for (size_t i = 0; i < heatMaxPosList.size(); ++i) {
    Long64_t currentHeatMaxPos = heatMaxPosList[i];

    // 对每个 heat 事件，查找附近 (±50) 的 light 事件
    for (size_t j = 0; j < lightMaxPosList.size(); ++j) {
        Long64_t currentLightMaxPos = lightMaxPosList[j];
        Long64_t delta = currentLightMaxPos - currentHeatMaxPos;

        // 如果 light 事件在 heat 事件的±50范围内
        // if (delta >= -50 && delta <= 50) {
        if (delta >= -100 && delta <= 50) {
            hLightDistribution->Fill(delta);  // 将光信号的时间差填入直方图
            fillCount++; // 计数
        }
    }
}

std::cout << "Number of filled bins: " << fillCount << std::endl;

// 定义高斯函数并指定拟合范围为20到40
TF1 *gausFit = new TF1("gausFit", "gaus", -40, -20);

// 对直方图进行高斯拟合
hLightDistribution->Fit(gausFit, "R");  // "R" 表示在指定范围内拟合

// 绘制 light 分布的直方图
TCanvas *canvas = new TCanvas("canvas", "Heat Events Distribution", 800, 600);
hLightDistribution->GetXaxis()->SetTitle("Time difference (Heat MaxPos - Light MaxPos)");
hLightDistribution->GetYaxis()->SetTitle("Counts/points");
canvas->cd();
hLightDistribution->Draw();
gausFit->Draw("same");  // 绘制高斯拟合结果

// 保存图像（可选）
canvas->SaveAs("Heat_Distribution.png");
// 关闭文件
heatFile->Close();
lightFile->Close();

Number of filled bins: 5222
 FCN=25.8611 FROM MIGRAD    STATUS=CONVERGED      81 CALLS          82 TOTAL
                     EDM=1.21738e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.06918e+01   2.14757e+00   3.66478e-03   4.59078e-06
   2  Mean        -3.05607e+01   4.45660e-01   1.11206e-03   3.28753e-05
   3  Sigma        6.81385e+00   6.61585e-01   6.41103e-05   2.56055e-04


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file Heat_Distribution.png has been created


Info in <TCanvas::Print>: png file Light_Distribution2.png has been created
